In [1]:
from flask import Flask, jsonify
import numpy as np
import pandas as pd
import mysql.connector

In [2]:
mysql_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'tugas_akhir',
}

conn = mysql.connector.connect(**mysql_config)

In [3]:
sql = "SELECT * FROM ( SELECT * FROM df_train_2 ORDER BY id DESC LIMIT 24 ) as SUBQUERY ORDER BY id"
cursor = conn.cursor(dictionary=True)
cursor.execute(sql)
myresult_prediction = cursor.fetchall()

df = pd.DataFrame(myresult_prediction)
df.drop(columns=['id'], inplace=True)

In [4]:
df

,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume,date_time
0,Not a Holiday,172.0,34.0,0.0,318.0,7.0,7.0,280.62,0.0,0.0,90.0,Clouds,overcast clouds,5215.0,2017-03-07 14:00:00
1,Not a Holiday,243.0,21.0,0.0,318.0,8.0,8.0,280.89,0.0,0.0,40.0,Clouds,scattered clouds,5709.0,2017-03-07 15:00:00
2,Not a Holiday,75.0,21.0,1.0,84.0,6.0,6.0,280.68,0.0,0.0,1.0,Rain,light rain,6722.0,2017-03-07 16:00:00
3,Not a Holiday,22.0,20.0,3.0,0.0,7.0,7.0,280.82,0.0,0.0,75.0,Rain,light rain,6017.0,2017-03-07 17:00:00
4,Not a Holiday,69.0,20.0,5.0,120.0,5.0,5.0,280.29,0.0,0.0,1.0,Clear,sky is clear,4775.0,2017-03-07 18:00:00
5,Not a Holiday,116.0,19.0,2.0,0.0,9.0,9.0,278.96,0.0,0.0,40.0,Clouds,scattered clouds,3425.0,2017-03-07 19:00:00
6,Not a Holiday,274.0,21.0,3.0,130.0,2.0,2.0,278.09,0.0,0.0,40.0,Clouds,scattered clouds,2815.0,2017-03-07 20:00:00
7,Not a Holiday,117.0,22.0,4.0,130.0,6.0,6.0,277.29,0.0,0.0,1.0,Clear,sky is clear,3727.0,2017-03-07 21:00:00
8,Not a Holiday,186.0,22.0,4.0,110.0,3.0,3.0,276.22,0.0,0.0,1.0,Clear,sky is clear,2816.0,2017-03-07 22:00:00
9,Not a Holiday,277.0,31.0,4.0,110.0,8.0,8.0,275.62,0.0,0.0,75.0,Clouds,broken clouds,1115.0,2017-03-07 23:00:00


# predict

In [5]:
import pandas as pd
import numpy as np
import joblib

In [6]:
encoder = joblib.load('outputs/shap/OHE_encoder_second.joblib')
scaler = joblib.load('outputs/shap/MM_scaler_second.joblib')
model = joblib.load('model/shap/BiLSTM_second.joblib')

Using TensorFlow backend.
c:\Users\Naufal\anaconda3\envs\shap_upgrade\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\Naufal\anaconda3\envs\shap_upgrade\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\Naufal\anaconda3\envs\shap_upgrade\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\Naufal\anacond

In [7]:
def preprocess_data(input_df):
    # Columns to convert to float
    columns_to_convert = [
        'air_pollution_index', 'humidity', 'wind_speed', 'wind_direction',
        'visibility_in_miles', 'dew_point', 'temperature', 'rain_p_h',
        'snow_p_h', 'clouds_all', 'traffic_volume'
    ]

    # Convert specified columns to float
    input_df[columns_to_convert] = input_df[columns_to_convert].astype(float)

    # Categorical Features
    ohe_column = ['is_holiday', 'weather_type', 'weather_description']
    encoded_data = encoder.transform(input_df[ohe_column])

    # Concatenate Encoded Columns to Main DataFrame
    df_concated = pd.concat([input_df, pd.DataFrame(encoded_data, columns=encoder.get_feature_names())], axis=1)
    df_concated.drop(columns=ohe_column, inplace=True)

    # Feature Engineering
    df_concated['date_time'] = pd.to_datetime(df_concated['date_time'])
    df_concated['day'] = df_concated['date_time'].dt.day
    df_concated['month'] = df_concated['date_time'].dt.month
    df_concated['hour'] = df_concated['date_time'].dt.hour

    # Set the 'date_time' as the index of the DataFrame
    df_concated.set_index('date_time', inplace=True)

    # Move Target column to the last
    df_concated['traffic_volume'] = df_concated.pop('traffic_volume')

    # Scale the data
    df_concated = pd.DataFrame(scaler.transform(df_concated), columns=df_concated.columns, index=df_concated.index)

    # Reshape into Input Data Requirement
    X = df_concated.values.reshape(1, 24, -1)  # Adjust the shape as per your model's requirement

    return X

def predict(input_df):
    X = preprocess_data(input_df)
    # X_copy = np.repeat(model.predict(X), 62, axis=-1)
    # X_unscale = scaler.inverse_transform(X_copy)[:,-1]
    return model.predict(X)

In [8]:
x = preprocess_data(df)

In [9]:
x

array([[[0.56055363, 0.21428571, 0.        , ..., 0.18181818,
         0.60869565, 0.71634615],
        [0.80622837, 0.05952381, 0.        , ..., 0.18181818,
         0.65217391, 0.7842033 ],
        [0.22491349, 0.05952381, 0.0625    , ..., 0.18181818,
         0.69565217, 0.92335165],
        ...,
        [0.11072664, 0.33333333, 0.375     , ..., 0.18181818,
         0.47826087, 0.65233516],
        [0.00692042, 0.30952381, 0.5       , ..., 0.18181818,
         0.52173913, 0.69642857],
        [0.13148789, 0.21428571, 0.6875    , ..., 0.18181818,
         0.56521739, 0.72980769]]])

In [10]:
joblib.dump(x, 'x.joblib')

['x.joblib']

In [16]:
predictt = predict(df)

In [17]:
predictt

array([[0.7502255]], dtype=float32)

In [18]:
from keras.models import load_model

In [19]:
modellls = load_model('model\shap\BiLSTM_second.h5')

AttributeError: 'str' object has no attribute 'decode'

In [22]:
import tensorflow as tf

In [23]:
tf.__version__

'1.14.0'

In [25]:
import keras

In [26]:
keras.__version__

'2.3.1'

In [28]:
!python --version

Python 3.6.6 :: Anaconda, Inc.
